In [13]:
%load_ext autoreload
%autoreload 2

import ee
import folium
import geemap
import gee_utils
# Authenticate to Earth Engine
ee.Authenticate()
# Initialize Earth Engine instance
ee.Initialize(project="reeftruth")
#!!! (for readme)
# https://stackoverflow.com/questions/78374548/no-authentication-box-appears-when-authenticating-google-earth-engine-gee-pyth


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Define functions

In [14]:
# Add Earth Engine drawing method to folium.
folium.Map.add_ee_layer = gee_utils.add_ee_layer

### Set visualisation parameters

In [27]:
paVisParams = {
  "palette": ["2ed033"],
  "min": 0.0,
  "max": 1550000.0,
  "opacity": 0.05
}
gdcrVisParams = {
  "palette": ["green"],
  "min": 0.0,
  "max": 1550000.0,
  "opacity": 0.8
}
wriVisParams = {
  "palette": ["red"],
  "min": 0.0,
  "max": 1550000.0,
  "opacity": 0.4
}
reefcheckVisParams = {
  "color": "white",  
  "pointSize": 200   
}

sentinelVisParams = {
  "min": 0,
  "max": 3000,
  "gamma": 1.5,
  "bands": ["B4", "B3", "B2"]
}

# centring and zoom
lat = -20
lon = 150
zoom = 10

## Add reef layers to map

TODO: detail layers

In [11]:
# import geemap

# # Create a map
# map = geemap.Map(center=[lat, lon], zoom=zoom)

# ### COPERNICUS
# # Load Copernicus image collection and filter by date
# copernicus = geemap.ee_imagecollection("COPERNICUS/S3/OLCI").filterDate("2018-02-02", "2018-02-03")

# # Select bands for visualization and apply band-specific scale factors.
# copernicus_image = copernicus.select(["Oa08_radiance", "Oa06_radiance", "Oa04_radiance"]) \
#                               .median() \
#                               .multiply([0.00876539, 0.0123538, 0.0115198])

# map.addLayer(copernicus_image, {}, "Copernicus background")

# ### ALLEN CORAL ATLAS
# aca_ds = geemap.ee_image("ACA/reef_habitat/v2_0")

# # Example mask application
# aca_reef_extent = aca_ds.select("reef_mask").selfMask()
# map.addLayer(aca_reef_extent, {"palette": ["EEEEEE"]}, "ACA Global reef extent")

# # Geomorphic zonation classification
# aca_geomorphic_zonation = aca_ds.select("geomorphic").selfMask()
# map.addLayer(aca_geomorphic_zonation, {}, "ACA Geomorphic zonation")

# # Benthic habitat classification
# aca_benthic_habitat = aca_ds.select("benthic").selfMask()
# map.addLayer(aca_benthic_habitat, {"palette": ["FF7F50"]}, "ACA Benthic habitat")

# ### UNEP-WCMC World Database of Coral Reefs (GDCR)
# unep_gdcr = geemap.ee_featurecollection("projects/reeftruth/assets/WCMC008_CoralReef2021_Py_v4_1")
# gdcr_image = geemap.ee_image().float().paint(unep_gdcr, "GDCR")
# map.addLayer(gdcr_image, {}, "UNEP-GDCR_polygons")

# ### World Resources Institute (WRI) Reefs at Risk
# wri_500 = geemap.ee_featurecollection("projects/reeftruth/assets/reef_500_poly")
# wri_image = geemap.ee_image().float().paint(wri_500, "WRI_500")
# map.addLayer(wri_image, {}, "WRI_500_polygons")

# ### Reef Check
# # reef_check = geemap.ee_featurecollection("projects/...")
# # map.addLayer(reef_check, {}, "Reef Check points")

# ### World Database on Protected Areas (WDPA)
# wdpa_ds = geemap.ee_featurecollection("WCMC/WDPA/current/polygons")
# wdpa_image = geemap.ee_image().float().paint(wdpa_ds, "REP_AREA")
# map.addLayer(wdpa_image, {}, "Protected areas")

# # Display the map
# map


In [39]:
# initialise map with bacgkround image
reef_map = geemap.Map(center=[lat, lon], zoom=zoom)

# Define the collections
sentinel_collection = ee.ImageCollection('COPERNICUS/S2')
cloud_collection = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')

# Filter the collections by date
sentinel_filtered = sentinel_collection.filterDate('2022-01-01', '2023-01-01')
cloud_filtered = cloud_collection.filterDate('2022-01-01', '2023-01-01')

# Link collections on common properties
sentinel_colour = sentinel_filtered.map(lambda image: image.addBands(
    cloud_filtered.filter(ee.Filter.equals('system:index', image.get('system:index'))).first()
))

# Define the cloud masking function
def cloud_mask(image):
    cloud_prob = image.select('probability')
    mask = cloud_prob.gt(0.6)
    return image.updateMask(mask)

# Apply the cloud masking function
sentinel_colour_masked = sentinel_colour.map(cloud_mask)

# Create the median composite
composite = sentinel_colour_masked.median()

reef_map.addLayer(composite, sentinelVisParams, 'sentinel_background');



### ALLEN CORAL ATLAS
aca_ds = ee.Image("ACA/reef_habitat/v2_0") 

# ACA reef extent
aca_reef_extent = aca_ds.select("reef_mask").selfMask()
reef_map.addLayer(aca_reef_extent, {"palette": ["EEEEEE"]}, "ACA Global reef extent")

# # Geomorphic zonation classification
# aca_geomorphic_zonation = aca_ds.select("geomorphic").selfMask()
# reef_map.addLayer(aca_geomorphic_zonation, {}, "ACA Geomorphic zonation")

# # Benthic habitat classification
# aca_benthic_habitat = aca_ds.select("benthic").selfMask()
# reef_map.addLayer(aca_benthic_habitat, {"palette": ["FF7F50"]}, "ACA Benthic habitat")

### World Resources Institute (WRI) Reefs at Risk
wri_500 = ee.FeatureCollection("projects/reeftruth/assets/reef_500_poly") 
wri_image = ee.Image().float().paint(wri_500, "WRI_500")
reef_map.addLayer(wri_image, wriVisParams, "WRI_500_polygons")



### UNEP-WCMC World Database of Coral Reefs (GDCR)
unep_gdcr = ee.FeatureCollection("projects/reeftruth/assets/WCMC008_CoralReef2021_Py_v4_1") 
gdcr_image = ee.Image().float().paint(unep_gdcr, "GDCR")
reef_map.addLayer(gdcr_image, gdcrVisParams, "UNEP-GDCR_polygons")


reef_map

Map(center=[-20, 150], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [16]:

# initialise map with bacgkround image
reef_map = folium.Map(location=[lat, lon], zoom_start=zoom)

### COPERNICUS
# Load Copernicus image collection and filter by date
copernicus = ee.ImageCollection("COPERNICUS/S3/OLCI").filterDate("2018-02-02", "2018-02-03")

# Select bands for visualization and apply band-specific scale factors.
copernicus_image = copernicus.select(["Oa08_radiance", "Oa06_radiance", "Oa04_radiance"]) \
                              .median() \
                              .multiply(ee.Image([0.00876539, 0.0123538, 0.0115198]))

reef_map.add_ee_layer(copernicus_image, copernicus_vps, "copernicus background")

### ALLEN CORAL ATLAS
aca_ds = ee.Image("ACA/reef_habitat/v2_0") 

# Example mask application
aca_reef_extent = aca_ds.select("reef_mask").selfMask()
reef_map.add_ee_layer(aca_reef_extent, {"palette": ["EEEEEE"]}, "ACA Global reef extent")

# Geomorphic zonation classification
aca_geomorphic_zonation = aca_ds.select("geomorphic").selfMask()
reef_map.add_ee_layer(aca_geomorphic_zonation, {}, "ACA Geomorphic zonation")

# Benthic habitat classification
aca_benthic_habitat = aca_ds.select("benthic").selfMask()
reef_map.add_ee_layer(aca_benthic_habitat, {"palette": ["FF7F50"]}, "ACA Benthic habitat")


### UNEP-WCMC World Database of Coral Reefs (GDCR)
unep_gdcr = ee.FeatureCollection("projects/reeftruth/assets/WCMC008_CoralReef2021_Py_v4_1") 
gdcr_image = ee.Image().float().paint(unep_gdcr, "GDCR")
reef_map.add_ee_layer(gdcr_image, gdcrVisParams, "UNEP-GDCR_polygons")

### World Resources Institute (WRI) Reefs at Risk
wri_500 = ee.FeatureCollection("projects/reeftruth/assets/reef_500_poly") 
wri_image = ee.Image().float().paint(wri_500, "WRI_500")
reef_map.add_ee_layer(wri_image, wriVisParams, "WRI_500_polygons")

### Reef Check
# # reef_check = ee.FeatureCollection("projects/...") 
# # reef_map.add_child(reef_check, reefcheck_vps, "Reef Check points")

### World Database on Protected Areas (WDPA)
wdpa_ds = ee.FeatureCollection("WCMC/WDPA/current/polygons")
wdpa_image = ee.Image().float().paint(wdpa_ds, "REP_AREA")
reef_map.add_ee_layer(wdpa_image, protectedareas_vps, "protected areas")

reef_map.add_child(folium.LayerControl())
display(reef_map)

In [11]:
ee.List([wdpa_ds.filter(ee.Filter.eq("geotype", "Polygon").geometry())])

TypeError: Filter.geometry() missing 1 required positional argument: 'geometry'

In [ ]:
# get basic info about a feature collection

def get_info(fc):
    """Can't just call .getInfo() on very large feature collections. This breaks it down"""
    print(f"Number of elements: {fc.size().getInfo()}")
    print(f"Type of first element: {fc.first().geometry().type().getInfo()}")
    print(f"Approximate size of first element: {fc.first().geometry().area().divide(1000000).getInfo()} km$^2$")
    return fc.getInfo()

def calculate_polygon_area(poly):
    """Calculate the area of a polygon in km^2"""
    return poly.area().divide(1000000).getInfo()

def calculate_feature_collection_area(fc):
    """Calculate the total area of a feature collection in km^2"""
    for poly in fc:
        print(calculate_polygon_area(poly))
    # return fc.geometry().area().divide(1000000).getInfo()

In [ ]:
study_region = ee.Geometry.Rectangle(130, -32, 170, 0)
# subset fc
gbr_wri_500 = wri_500.filterBounds(study_region)
gbr_wdpa_ds = wdpa_ds.filterBounds(study_region)


In [ ]:
wri_500.size().getInfo()

In [ ]:
geoms = wri_500.geometry().geometries()
features = ee.FeatureCollection(geoms.map(lambda geo: ee.Feature(geo)))

In [ ]:
features.first().getInfo()

In [ ]:
wri_500.first().getInfo()

In [ ]:
wri_500.type().getInfo()

In [ ]:
wri_500.getInfo()

In [ ]:
wdpa_ds.size().getInfo()

In [ ]:
gbr_wdpa_ds.size().getInfo()

In [ ]:
wdpa_ds.first().geometry().area().divide(1000000).getInfo()

In [ ]:
# wdpa_ds.union().geometry().getInfo()
wri_500.first().getInfo()